### Install Gradio for the interactive web interface


In [ ]:
!pip install gradio --quiet

# Import necessary libraries

In [ ]:
from transformers import pipeline

import io
from io import BytesIO

from PIL import Image
import requests

from bs4 import BeautifulSoup

import gradio as gr

# Loading pre-trained model from hugging face

In [ ]:
# Load pre-trained model for image captioning (by Salesforce)
pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

# 1. Generate Caption from Image


In [ ]:
# Function to generate a text caption from an uploaded image
def generate_caption(image):
    try:
        caption = pipe(image)[0]['generated_text']
        return caption
    except Exception as e:
        return f"An error occurred: {e}"

# 2. Image Search Function


In [ ]:
# Fetches image URLs by performing a Google Image search based on text
def google_image_search_from_text(search_term, num_images=10):
    try:
        search_url = f"https://www.google.com/search?q={search_term}&tbm=isch"
        response = requests.get(search_url)
        response.raise_for_status()  # Ensure request is successful

        soup = BeautifulSoup(response.content, "html.parser")
        img_tags = soup.find_all("img")

        image_urls = []
        for i, img_tag in enumerate(img_tags):
            if i >= num_images+1:  # Limit search to first few results
                break
            if 'src' in img_tag.attrs:
                image_urls.append(img_tag['src'])
        return image_urls
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# 3. Read Image from URL


In [ ]:
# Fetches and opens an image from a URL for processing
def read_image_url(image_url):
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  # Ensure successful download
        image = Image.open(BytesIO(response.content))
        return image
    except requests.exceptions.RequestException as e:
        return f"Error downloading image: {e}"
    except Exception as e:
        return f"An error occurred: {e}"

# 4. Search Similar Images


In [ ]:
# Using the caption, performs a similar image search and retrieves matching results
def search_similar_images(image):
    text = generate_caption(image)
    try:
        urls = google_image_search_from_text(text)
        returned_images = []

        for url in urls[1:]:  # Skip the first image (usually irrelevant)
            try:
                returned_images.append(read_image_url(url))
            except:
                continue
        return returned_images
    except:
        return []

# 5. Gradio Interface Setup

In [ ]:
# Setting up the Gradio interface for easy user interaction
with gr.Blocks() as demo:
    gr.Markdown("## Visual Search App: Upload an Image to Find Similar Ones")

    # Upload and button row
    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload an Image")
        search_button = gr.Button("Search")

    # Display results in a gallery format
    result_gallery = gr.Gallery(label="Search Results", show_label=True)

    # Connect search function to button and gallery output
    search_button.click(fn=search_similar_images, inputs=image_input, outputs=result_gallery)


In [ ]:
# Launch the Gradio app for interactive exploration
demo.launch()